In [1]:
!pip install pymorphy2[fast]

Импортируем библоитеки

In [2]:
from google.colab import drive
import pandas as pd
from nltk.stem.snowball import RussianStemmer
from pymorphy2 import MorphAnalyzer
import numpy as np
import torch.nn as nn
import torch
import multiprocessing
from gensim.models import Word2Vec
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
import torch.optim as optim
import time
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

#Часть 1. Чтение данных и первичная предобработка

В программе будет три режима работы:
1.   Без уменьшения словаря
2.   Стемминг
3.   Лемматизация

In [3]:
REDUCTION_MODE = 2 #0 - no reduction, 1 - stemming, 2 - lemmatize

Подключаем гугл диск, где содержится файл с корпусом, который был получен ранее

In [4]:
drive.mount('/content/drive/')

dir = 'drive/MyDrive/BS/DATA_EXTRACTION/'
corp_cased = dir + 'corp_cased.csv'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Считываем csv файл корпуса и выводим первые пять элементов

In [5]:
df = pd.read_csv(corp_cased, sep='\t', header=None, on_bad_lines='skip')

df.head()

,0,1
0,Школа злословия учит прикусить язык,NOUN NOUN VERB INFN NOUN
1,Сохранится ли градус дискуссии в новом сезоне,VERB PRCL NOUN NOUN PREP ADJF NOUN
2,Великолепная Школа злословия вернулась в эфир ...,ADJF NOUN NOUN VERB PREP NOUN PREP ADJF NOUN P...
3,В истории программы это уже не первый ребрендинг,PREP NOUN NOUN NPRO ADVB PRCL ADJF NOUN
4,Сейчас с трудом можно припомнить что начиналас...,ADVB PREP NOUN PRED INFN CONJ VERB NOUN PREP N...


Удаялем строки с nan значениями

In [6]:
df.dropna(inplace=True)

In [7]:
stemmer = RussianStemmer()
lemmatizer = MorphAnalyzer()

Определяем функцию, которая исходя из режима работы делает соотвестующие преобразования слов

In [8]:
def reduction(x):
    x = str(x)
    if REDUCTION_MODE == 0:
        return x.split()
    elif REDUCTION_MODE == 1:
        return [stemmer.stem(token) for token in x.split(' ')]
    elif REDUCTION_MODE == 2:
        return [lemmatizer.normal_forms(token)[0] for token in x.split(' ')]

Используем функцию выше

In [9]:
sentences = df[0].to_numpy()

sentences = np.array(list(map(reduction, sentences)))

print(sentences[:5])

[list(['школа', 'злословие', 'учить', 'прикусить', 'язык'])
 list(['сохраниться', 'ли', 'градус', 'дискуссия', 'в', 'новый', 'сезон'])
 list(['великолепный', 'школа', 'злословие', 'вернуться', 'в', 'эфир', 'после', 'летний', 'каникулы', 'в', 'новый', 'формат'])
 list(['в', 'история', 'программа', 'это', 'уже', 'не', 'первый', 'ребрендинг'])
 list(['сейчас', 'с', 'труд', 'можно', 'припомнить', 'что', 'начинаться', 'школа', 'на', 'канал', 'культура', 'как', 'стандартный', 'ток-шоу', 'который', 'отличаться', 'от', 'другой', 'кухонный', 'обсуждение', 'гость', 'что', 'называться', 'за', 'глаз', 'и', 'неожиданный', 'персона', 'в', 'качество', 'ведущий'])]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


Также разделяем теги на токены

In [10]:
tags = df[1].to_numpy()

tags = np.array(list(map(lambda x: str(x).split(), tags)))

print(tags[:5])

[list(['NOUN', 'NOUN', 'VERB', 'INFN', 'NOUN'])
 list(['VERB', 'PRCL', 'NOUN', 'NOUN', 'PREP', 'ADJF', 'NOUN'])
 list(['ADJF', 'NOUN', 'NOUN', 'VERB', 'PREP', 'NOUN', 'PREP', 'ADJF', 'NOUN', 'PREP', 'ADJF', 'NOUN'])
 list(['PREP', 'NOUN', 'NOUN', 'NPRO', 'ADVB', 'PRCL', 'ADJF', 'NOUN'])
 list(['ADVB', 'PREP', 'NOUN', 'PRED', 'INFN', 'CONJ', 'VERB', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'CONJ', 'ADJF', 'NOUN', 'ADJF', 'VERB', 'PREP', 'ADJF', 'ADJF', 'NOUN', 'NOUN', 'CONJ', 'VERB', 'PREP', 'NOUN', 'CONJ', 'ADJF', 'NOUN', 'PREP', 'NOUN', 'ADJF'])]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


Создаем Word2Vec модель из имеющихся предложений

In [11]:
size, window, min_cnt, sg = 30, 2, 2, 0 # Используем модель CBOW
workers = multiprocessing.cpu_count()
n_iter = 150
w2v_model = Word2Vec(sentences, size = size, window = window, min_count = min_cnt,
                    sg = sg, workers = workers, iter = n_iter)

# Часть 2. Подготовка датасетов для моделей

Создаем функции для создания словарей слов и тегов (чтобы перевести текст в цифру), а также создаем модель для классификатора\
[слово_до, слово, слово_после] -> часть речи

In [12]:
def build_voc_w(stoi):
    idx = 1

    for sentence in sentences:
        for word in sentence:
            if word not in stoi:
                stoi[word] = idx
                idx += 1


def build_voc_t(ttoi):
    idx = 0
    
    for tags_ in tags:
        for tag in tags_:
            if tag not in ttoi:
                ttoi[tag] = idx
                idx += 1

def creator(x, y, stoi, ttoi):
    for i in range(len(sentences)):
        for j in range(len(sentences[i])):
            x_elem = []
            #word before
            if j == 0:
                x_elem.append(0)
            else:
                x_elem.append(stoi[sentences[i][j - 1]])

            #current word
            x_elem.append(stoi[sentences[i][j]])

            #word after
            if j == len(sentences[i]) - 1:
                x_elem.append(0)
            else:
                x_elem.append(stoi[sentences[i][j + 1]])

            x.append(x_elem)
            y.append(ttoi[tags[i][j]])

Применяем определенные выше функции. Нулевой индекс оставляем для выравнивания

In [13]:
#sentences vocs
stoi = {None: 0}

#tags vocs
ttoi = {}

build_voc_w(stoi)
build_voc_t(ttoi)

x = []
y = []

creator(x, y, stoi, ttoi)

Пример данных

In [14]:
print(x[:5])
print(y[:5])

[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 0]]
[0, 0, 1, 2, 0]


Определяем устройство на котором будет обучаться модель

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Переопределяем класс Dataset из torch

In [16]:
class PosTagDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

        self.x = torch.LongTensor(self.x).to(device)
        self.y = torch.LongTensor(self.y).to(device)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

Создаем функцию для разделения данных, указываем параметр stratify для сбалансированности классов

In [23]:
def train_val_test_split(x, y, train_size=0.7, val_size=0.1):
    #test = 1 - train - val
    x_train, x_, y_train, y_ = train_test_split(x, y, train_size=train_size, stratify=y, shuffle=True)
    x_val, x_test, y_val, y_test = train_test_split(x_, y_, train_size=val_size/(1-train_size), stratify=y_, shuffle=True)

    return x_train, y_train, x_val, y_val, x_test, y_test

Делим данные на три множества: train, validation, test

In [24]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x, y)

Создаем датасеты

In [25]:
dataset_train = PosTagDataset(x_train, y_train)
dataset_val = PosTagDataset(x_val, y_val)
dataset_test = PosTagDataset(x_test, y_test)

Проверяем корректность метода '__getitem __'

In [26]:
print(dataset_train.__getitem__(0))

(tensor([814,  23,  10]), tensor(0))


Создаем DataLoader на основе созданные ранее датасетов

In [27]:
batch_size = 64

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

Определяем функцию для получения весов из W2V модели, чтобы в дальнейшем

In [28]:
#перевод word2vec в массив весов для слоя Embedding    
def make_e_weights():
    # wv.index2word - список слов словаря
    # wv.vectors - массив координат слов
    dict_w2v = dict(zip(w2v_model.wv.index2word, w2v_model.wv.vectors))
    e_weights = np.zeros((len(stoi), size))
    for w, t in stoi.items(): # Слово и его код
        w_coords = dict_w2v.get(w) # Координаты слова
        if w_coords is not None:
            e_weights[t] = w_coords
    return torch.FloatTensor(e_weights)

Определяем модель

In [29]:
class W2VPoSTagger(nn.Module):
    def __init__(self, size_w2v, hidden_layer_s):
        
        super().__init__()
        
        weights = make_e_weights()
        weights.to(device)
        self.embedding = nn.Embedding.from_pretrained(weights, freeze=False)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(size_w2v * 3, hidden_layer_s)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_layer_s, len(ttoi))
        self.act2 = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        #x = self.act2(x)

        return x

Создаем экземпляр модели

In [30]:
model = W2VPoSTagger(size_w2v=size, hidden_layer_s=30)
print(model)

W2VPoSTagger(
  (embedding): Embedding(66179, 30)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=90, out_features=30, bias=True)
  (act1): ReLU()
  (fc2): Linear(in_features=30, out_features=22, bias=True)
  (act2): Softmax(dim=1)
)


Считаем количество параметров

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Модель имеет {count_parameters(model):,} обучаемых параметров')

Модель имеет 1,988,782 обучаемых параметров


# Часть 3. Обучение модели

Определяем оптимизатор и функцию потерь

In [32]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

Помещаем модель и функцию потерь на `device`

In [33]:
model = model.to(device)
criterion = criterion.to(device)

Определяем функцию обучения модели

In [34]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0

    model.train()

    all_preds = []
    all_tags = []
    
    for batch in iterator:
        
        text = batch[0]
        tags = batch[1]
                
        optimizer.zero_grad()
        
        predictions = model(text)

        all_preds.append(predictions.detach().cpu().numpy())
        all_tags.append(tags.detach().cpu().numpy())

        loss = criterion(predictions, tags)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

        
    return epoch_loss / len(iterator), np.concatenate(all_preds, 0).argmax(1).reshape(-1), np.concatenate(all_tags, 0)

Определяем функцию валидации модели

In [35]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    
    model.eval()

    all_preds = []
    all_tags = []
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch[0]
            tags = batch[1]
            
            predictions = model(text)

            all_preds.append(predictions.detach().cpu().numpy())
            all_tags.append(tags.detach().cpu().numpy())
            
            loss = criterion(predictions, tags)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator), np.concatenate(all_preds, 0).argmax(1).reshape(-1), np.concatenate(all_tags, 0)

Определяем функцию для подсчета времени выполнения

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Задаем параметры для `classification_report`

In [37]:
cr_labels = []
cr_names = []

for name, label in ttoi.items():
    cr_labels.append(label)
    cr_names.append(name)

print(cr_labels)
print(cr_names)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
['NOUN', 'VERB', 'INFN', 'PRCL', 'PREP', 'ADJF', 'NPRO', 'ADVB', 'PRED', 'CONJ', 'Name', 'Surn', 'PRTF', 'COMP', 'NUMR', 'UNKN', 'Patr', 'INTJ', 'PRTS', 'GRND', 'Geox', 'ADJS']


Обучаем модель

In [38]:
N_EPOCHS = 15

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_preds, train_tags = train(model, dataloader_train, optimizer, criterion)
    valid_loss, _, __ = evaluate(model, dataloader_val, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')

    print(classification_report(train_tags, train_preds, labels=cr_labels, target_names=cr_names))

    print(f'\t Val Loss: {valid_loss:.3f}')

Epoch: 01 | Epoch Time: 11m 57s
	Train Loss: 0.546
              precision    recall  f1-score   support

        NOUN       0.88      0.93      0.90    356194
        VERB       0.69      0.86      0.76     96780
        INFN       0.69      0.46      0.55     25493
        PRCL       0.88      0.85      0.87     32664
        PREP       0.97      0.98      0.97    130292
        ADJF       0.82      0.85      0.84    163014
        NPRO       0.86      0.84      0.85     32607
        ADVB       0.79      0.71      0.75     41082
        PRED       0.85      0.67      0.75      3576
        CONJ       0.92      0.95      0.94     96608
        Name       0.62      0.51      0.56     12913
        Surn       0.58      0.41      0.48      9474
        PRTF       0.46      0.12      0.19     17090
        COMP       0.60      0.27      0.37      2589
        NUMR       0.87      0.76      0.81      5613
        UNKN       0.48      0.38      0.43     23719
        Patr       0.71      0

Тестируем модель на отложенной выборке

In [39]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_preds, test_tags = evaluate(model, dataloader_test, criterion)

print(f'Test Loss: {test_loss:.3f}')

print(test_preds[10:])
print(test_tags[10:])

print(classification_report(test_tags, test_preds, labels=cr_labels, target_names=cr_names))

Test Loss: 0.209
[ 5 18 15 ...  0  0  5]
[ 5 18 15 ...  0  0  5]
              precision    recall  f1-score   support

        NOUN       0.98      0.97      0.98    101770
        VERB       0.78      0.94      0.85     27652
        INFN       0.87      0.60      0.71      7284
        PRCL       0.95      0.94      0.94      9333
        PREP       0.99      1.00      0.99     37226
        ADJF       0.94      0.96      0.95     46575
        NPRO       0.94      0.92      0.93      9316
        ADVB       0.96      0.93      0.95     11738
        PRED       0.92      0.88      0.90      1022
        CONJ       0.98      0.99      0.98     27602
        Name       0.91      0.86      0.88      3689
        Surn       0.95      0.86      0.90      2707
        PRTF       0.71      0.47      0.56      4883
        COMP       0.73      0.76      0.75       739
        NUMR       0.95      0.99      0.97      1603
        UNKN       0.73      0.77      0.75      6777
        Patr    

# Часть 4. Использование модели keras

Импортируем необходимые библиотеки

In [40]:
import keras
from keras import losses
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Embedding, Dropout, Reshape
import time

import tensorflow
from tensorflow.math import argmax

In [41]:
print(x[:10])
print(y[:10])

[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 0], [0, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9, 10], [9, 10, 11]]
[0, 0, 1, 2, 0, 1, 3, 0, 0, 4]


Создаем модель

In [42]:
#создание модели          
def create_model(sq, num_words, size, num_classes):
    inp = Input(shape = (sq, ), dtype = 'int32')
    e_weights = make_e_weights()
    x = Embedding(num_words, output_dim = size, input_length = sq, 
                weights = [e_weights], trainable = True)(inp)
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    x = Dense(size, activation = 'relu', use_bias = True)(x)
    output = Dense(num_classes, activation = 'softmax', use_bias = True)(x)
    model = Model(inp, output)
    model.summary()
    return model

Определяем функцию-обертку для `clasification_report`

In [43]:
class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.data = []

    def on_epoch_end(self, batch, logs={}):
        print(self.validation_data)
        X_val, y_val = self.batch[0], self.batch[1]
        y_predict = model.predict(X_val)

        y_predict = argmax(y_predict, 1)

        self.data.append(classification_report(y_val.numpy(), y_predict.numpy(), labels=cr_labels, target_names=cr_names))

    def get_data(self):
        return self.data

Обучаем модель

In [44]:
#обучение НС
model = create_model(3, len(stoi), 30, len(ttoi))

model.compile(optimizer = 'adam', loss = losses.sparse_categorical_crossentropy, metrics = ['accuracy'], run_eagerly=True)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

startTime = time.time()
metrics = Metrics()
history = model.fit(x_train, y_train, batch_size = 128, epochs = 15, verbose = 2, validation_data = (x_test, y_test))
print('Full time:', time.time() - startTime)

pred = model.predict(x_test, batch_size=32, verbose=2)
predicted = np.argmax(pred, axis=1)
report = classification_report(y_test, predicted, labels=cr_labels, target_names=cr_names)
print(report)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 embedding (Embedding)       (None, 3, 30)             1985370   
                                                                 
 flatten (Flatten)           (None, 90)                0         
                                                                 
 dropout (Dropout)           (None, 90)                0         
                                                                 
 dense (Dense)               (None, 30)                2730      
                                                                 
 dense_1 (Dense)             (None, 22)                682       
                                                                 
Total params: 1,988,782
Trainable params: 1,988,782
Non-train